<a href="https://colab.research.google.com/github/fhfmendes04/HA_FJSSP/blob/main/HA_AG_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from copy import deepcopy


In [33]:
class individuo():
  def __init__(self, job, operation, machine, regras_MS):
    self.OS = []
    self.MS = []
    self.regras_MS = regras_MS

    self.job = job
    self.operation = operation
    self.machine = machine

    self.OS_String()
    self.MS_String()

    self.scheduling = []
    self.makespan = 0

  def OS_String(self):
    lista = [3,3,3,3,3,3,3,3,3,3,3,3]
    for l in range(0,self.job * self.operation):
      flag = True
      while flag:
        numero = random.randint(0,11)
        if lista[numero]>=0:
          lista[numero] -= 1
          self.OS.append(numero)
          flag=False

  def MS_String(self):
    for j in range(0, self.job):
      reg = self.regras_MS[j]
      for o in range(0, self.operation):
        self.MS.append(random.choice(reg[o]))

In [130]:
from pandas._config.config import pp_options_list
class HA_AG_TS():
  def __init__(self, tamanho_populacao=400, geracao=200, prob_reproduction = 0.5, prob_crossover=0.8, prob_mutacao=0.1):
    self.tamanho_populacao = tamanho_populacao
    self.populacao = []
    self.geracao = geracao
    self.prob_crossover = prob_crossover
    self.prob_mutacao = prob_mutacao
    self.prob_reproduction = prob_reproduction
    self.lista_solucoes = []
    self.df = pd.DataFrame()
    self.job = 0
    self.operation = 0
    self.machine = 0
    self.regras_MS = []
    self.gen = 0
    self.pop_ordenada = []
    self.parents = []
    self.offsprings = []
    self.best = []
    self.tamanho =  0

    self.upload()
    self.regramento_MS()

  def upload(self):
      arquivo = "/content/drive/MyDrive/USP/Mestrado Sistema de Informação/SIN 5006 - Inteligência Computacional/Apresentação I/code/MFJS01.xlsx"
      self.df = pd.read_excel(arquivo, index_col=False)
      self.job = len(self.df['JOB'].unique())
      self.operation = len(self.df['OPERATION'].unique())
      self.machine = len(self.df['MACHINE'].unique())
      print("JOB: ", self.job)
      print("OPERATION: ", self.operation)
      print("MACHINE: ", self.machine)

      self.df.fillna(0,inplace=True)
      self.df["MACHINE.2"] = self.df["MACHINE.2"].astype(int)
      self.df['TIME.2'] = self.df["TIME.2"].astype(int)

  def regramento_MS(self):
    for j in range(0,self.job):
      regra_o = []
      dados = self.df[self.df["JOB"]==j].reset_index(drop=True)
      for o in range(0, self.operation):
        regra = []
        machine_1 = dados.iloc[o][3]
        machine_2 = dados.iloc[o][5]
        machine_3 = dados.iloc[o][7]

        if (machine_1 != 0):
          regra.append(machine_1)
        elif (machine_1 == 0):
          if (dados.iloc[0][4]!= 0):
            regra.append(machine_1)

        if(machine_2 != 0):
          regra.append(machine_2)
        elif (machine_2 == 0):
          if(dados.ilo[o][6] != 0):
            regra.append(machine_2)

        if(machine_3 !=0):
          regra.append(machine_3)
        elif (machine_3 == 0):
          if (dados.iloc[o][8] !=0):
            regra.append(machine_3)

        regra_o.append(regra)

      self.regras_MS.append(regra_o)

  def fitness(self,os_lista, ms_lista):
    maquinas = []
    controle_os = np.zeros(self.job)

    #Transforma a lista de 48 elementos em 12 x 4
    conta = 0
    mostra = []
    maquinas = []
    machine_job = [[0],[0],[0],[0],[0],[0],[0],[0]]
    job_time = [[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]]
    for i in ms_lista:
      mostra.append(i)
      conta +=1
      if ((conta % 4) == 0):
        conta = 0
        maquinas.append(mostra)
        mostra=[]
    #print(maquinas)
    for o in os_lista:
      posicao = int(controle_os[o])
      controle_os[o] += 1
      #print(o,posicao, controle_os, maquinas[o][posicao])
      dados = self.df[self.df["JOB"]==o].reset_index(drop=True)
      localizador = maquinas[o][posicao]
      machine_1 = dados.iloc[posicao][3]
      machine_2 = dados.iloc[posicao][5]
      machine_3 = dados.iloc[posicao][7]

      if (machine_1 == localizador):
        if (dados.iloc[posicao][4]!= 0):
          if (posicao == 0): # Primeira vez na machine
            machine_job[localizador].append("O(" + str(o) + str(posicao) + "):" + str(machine_job[localizador][0]) + "-" + str(machine_job[localizador][0] + dados.iloc[posicao][4]))
            machine_job[localizador][0] = machine_job[localizador][0] + dados.iloc[posicao][4]
            job_time[o] = machine_job[localizador][0]
          else:
              inicio = job_time[o]
              final = job_time[o] + dados.iloc[posicao][4]
              ultimo = final
              for m in range(1, len(machine_job[localizador])):
                  s = machine_job[localizador][m]
                  index_1 = s.find(":")
                  index_2 = s.find("-")
                  numero_1 = int(s[(index_1+1):index_2])
                  numero_2 = int(s[(index_2+1):])
                  if (((inicio > numero_1) and (inicio >= numero_2)) and ((final > numero_1) and (final > numero_2))):
                      continue
                  else:
                    if ((inicio < numero_1) and (final < numero_1) and (ultimo >= inicio)):
                        break
                    else:
                      if ((inicio >= numero_1) and (inicio >= numero_2)):
                        break
                      else:
                        inicio = numero_2
                        ultimo = final
                        final = inicio + dados.iloc[posicao][4]

              machine_job[localizador].append("O(" + str(o) + str(posicao) + "):" + str(inicio) + "-" + str(final))
              machine_job[localizador][0] = final
              job_time[o] = machine_job[localizador][0]
      else:
        if (machine_2 == localizador):
          if (dados.iloc[posicao][6]!= 0):
            if (posicao == 0): # Primeira vez na machine
              machine_job[localizador].append("O(" + str(o) + str(posicao) + "):" + str(machine_job[localizador][0]) + "-" + str(machine_job[localizador][0] + dados.iloc[posicao][4]))
              machine_job[localizador][0] = machine_job[localizador][0] + dados.iloc[posicao][4]
              job_time[o] = machine_job[localizador][0]
            else:
              inicio = job_time[o]
              final = job_time[o] + dados.iloc[posicao][4]
              ultimo = final
              for m in range(1, len(machine_job[localizador])):
                  s = machine_job[localizador][m]
                  index_1 = s.find(":")
                  index_2 = s.find("-")
                  numero_1 = int(s[(index_1+1):index_2])
                  numero_2 = int(s[(index_2+1):])
                  if (((inicio > numero_1) and (inicio >= numero_2)) and ((final > numero_1) and (final > numero_2))):
                      continue
                  else:
                    if ((inicio < numero_1) and (final < numero_1) and (ultimo >= inicio)):
                        break
                    else:
                      if ((inicio >= numero_1) and (inicio >= numero_2)):
                        break
                      else:
                        inicio = numero_2
                        ultimo = final
                        final = inicio + dados.iloc[posicao][4]

              machine_job[localizador].append("O(" + str(o) + str(posicao) + "):" + str(inicio) + "-" + str(final))
              machine_job[localizador][0] = final
              job_time[o] = machine_job[localizador][0]
        else:
          if (machine_3 == localizador):
            if (dados.iloc[posicao][8]!= 0):
              if (posicao == 0): # Primeira vez na machine
                machine_job[localizador].append("O(" + str(o) + str(posicao) + "):" + str(machine_job[localizador][0]) + "-" + str(machine_job[localizador][0] + dados.iloc[posicao][4]))
                machine_job[localizador][0] = machine_job[localizador][0] + dados.iloc[posicao][4]
                job_time[o] = machine_job[localizador][0]
              else:
                inicio = job_time[o]
                final = job_time[o] + dados.iloc[posicao][4]
                ultimo = final
                for m in range(1, len(machine_job[localizador])):
                  s = machine_job[localizador][m]
                  index_1 = s.find(":")
                  index_2 = s.find("-")
                  numero_1 = int(s[(index_1+1):index_2])
                  numero_2 = int(s[(index_2+1):])
                  if (((inicio > numero_1) and (inicio >= numero_2)) and ((final > numero_1) and (final > numero_2))):
                    continue
                  else:
                    if ((inicio < numero_1) and (final < numero_1) and (ultimo >= inicio)):
                      break
                    else:
                      if ((inicio >= numero_1) and (inicio >= numero_2)):
                        break
                      else:
                        inicio = numero_2
                        ultimo = final
                        final = inicio + dados.iloc[posicao][4]

                machine_job[localizador].append("O(" + str(o) + str(posicao) + "):" + str(inicio) + "-" + str(final))
                machine_job[localizador][0] = final
                job_time[o] = machine_job[localizador][0]

    maior = 0
    for l in machine_job:
      if l[0] > maior:
        maior = l[0]
    return maior, machine_job


  def initialize_population(self):
    self.gen = 1
    print(f"\nInitialize the Population({self.gen}).")
    for i in tqdm(range(self.tamanho_populacao)):
      self.populacao.append(individuo(self.job, self.operation, self.machine, self.regras_MS))

  def evaluate_population(self):
    print(f"\nEvaluate the Population({self.gen}).")
    for p in tqdm(self.populacao):
      p.makespan, p.scheduling  =  self.fitness(p.OS, p.MS)
    self.pop_ordenada = sorted(self.populacao,
                            key = lambda populacao: populacao.makespan,
                            reverse = False)
    self.best = self.pop_ordenada[0]
    print("\nMakerspan:", self.pop_ordenada[0].makespan)


  def selection(self, popular):
    self.tamanho = int(self.prob_reproduction * self.tamanho_populacao)
    escolha = random.randint(0, 1)
    if escolha == 0: #Elitist Selection
      print(f"\nSelection: Elitist({self.tamanho}).")
      self.parents =  deepcopy(self.pop_ordenada[0:self.tamanho])
      self.offsprings =  deepcopy(self.pop_ordenada[0:self.tamanho])
    else: # Tournament Selection
      print(f"\nSelection: Tournament({self.tamanho})")
      self.parents =  deepcopy(random.choices(popular, k=self.tamanho))
      self.offsprings =  deepcopy(random.choices(popular, k=self.tamanho))

  #def POX(self, pai1, pai2):


  #def JBX(self, pai1, pai2):
  '''def crossover(self, popular):
    escolha = random.randint(0, 1)
    if escolha == 0: #POX


    else: #JBX
'''
  def soluction(self):
    self.initialize_population()
    self.evaluate_population()
    self.selection(HA.populacao)
    while self.gen < self.geracao:
      self.gen += 1
      contador = self.tamanho
      while contador < self.tamanho_populacao:
        pais = deepcopy(random.choices(self.parents, k=2))
        pai1 = pais[0]
        pai2 = pais[1]
        filhos_c = self.crossover(pai1, pai2)
        #filhos_m = self.mutacao(pa1,pa12)






In [132]:
HA = HA_AG_TS()
HA.soluction()


JOB:  12
OPERATION:  4
MACHINE:  8

Initialize the Population(1).


100%|██████████| 400/400 [00:00<00:00, 8005.20it/s]



Evaluate the Population(1).


100%|██████████| 400/400 [00:14<00:00, 27.90it/s]


Makerspan: 1391

Selection: Elitist(200).


In [117]:
len(HA.parents)

200

In [103]:
print(HA.populacao[10].OS)
print(HA.populacao[10].MS)
print(HA.populacao[10].makespan)
print(HA.populacao[10].scheduling)


[8, 6, 2, 5, 0, 9, 0, 10, 10, 10, 6, 9, 5, 4, 0, 5, 5, 7, 9, 2, 1, 11, 0, 9, 10, 11, 3, 3, 7, 8, 11, 3, 2, 7, 8, 7, 3, 11, 6, 8, 6, 4, 1, 2, 4, 4, 1, 1]
[2, 6, 6, 7, 0, 2, 5, 4, 1, 2, 5, 5, 1, 4, 5, 4, 2, 4, 1, 7, 3, 4, 6, 6, 2, 4, 4, 4, 2, 4, 6, 6, 2, 3, 6, 5, 2, 4, 6, 7, 1, 3, 4, 5, 1, 2, 6, 7]
2326
[[214, 'O(10):0-214'], [2161, 'O(20):0-87', 'O(100):87-237', 'O(110):237-582', 'O(30):582-669', 'O(42):2051-2161'], [1611, 'O(80):0-150', 'O(60):150-295', 'O(00):295-542', 'O(90):542-787', 'O(40):787-910', 'O(70):910-1155', 'O(21):1155-1300', 'O(111):1300-1524', 'O(11):1524-1611'], [597, 'O(50):0-145', 'O(101):237-417', 'O(81):417-597'], [2171, 'O(102):417-457', 'O(61):457-581', 'O(91):787-1011', 'O(51):145-295', 'O(31):1011-1261', 'O(71):1261-1485', 'O(33):1485-1735', 'O(62):581-726', 'O(63):1735-1965', 'O(41):1965-2051', 'O(13):2051-2171'], [1789, 'O(103):457-607', 'O(32):1261-1406', 'O(22):1406-1596', 'O(83):1854-2004', 'O(23):1596-1766', 'O(12):1611-1789'], [2004, 'O(01):542-682', 'O(